## Conformal learning from scratch

Conformal learning is a framework that allows constructing predictions with predefined accuracy guarantees for *iid* data. It can be used on top of any traditional predictive algorithm for both classification and regression. The theory behind conformal learning might seem complicated, but it is based on a simple idea.

The goal of this notebook is to guide you through the theory of conformal regressors.

We will build a conformal regressors step-by-step explaining the required background on the way. In the end, we will confirm that the constructed model gives the same results as the code from nonconformist library.

We will be working with the Inductive Conformal Prediction (ICP) framework, because conformal regressor has no transductive algorithm.

Very first step: importing required libraries.

In [2]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes
from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
from nonconformist.nc import SignErrorErrFunc
from nonconformist.nc import AbsErrorErrFunc

For demonstration, we will use a California Housing dataset from [sklearn library](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) .

For future computatuion we should split the data into fractions - training and test sets. Additionaly, training data will be splitted into training and calibration sets. But for current section we will use only test and training sets. Calibration one will be used in the next sections.

In [3]:

data_data, data_target = fetch_california_housing(return_X_y= True)

df_features = pd.DataFrame(data_data)
df_target = pd.DataFrame(data_target)
idx_size = df_target.size

np.random.seed(2)

idx = np.random.permutation(len(data_data))

# test = 10%, test(test+calib) = 90% 
test_size = int(idx_size*0.1)
train_size = idx_size-test_size
calib_size = 2000
train_size = train_size - calib_size

idx_train, idx_cal, idx_test = idx[:train_size], idx[train_size:train_size + calib_size], idx[train_size + calib_size:]


print('Test size: {}'.format(test_size))
print('Calibration size: {}'.format(calib_size))
print('Train size: {}'.format(train_size))

Test size: 2064
Calibration size: 2000
Train size: 16576


After splitting the data we need to feed the model with training data. For our example *Random Forest Regressor* was chosen. After our model ate some data-food we can start generating predictions.

In [4]:
model = RandomForestRegressor()
model.fit(data_data[idx_train, :], data_target[idx_train])
predictions_cal = model.predict(data_data[idx_cal,:])
predictions_test = model.predict(data_data[idx_test,:])

A **nonconformity** functions are used to measure the nonconformity or strageness of the data point. For regression example there're two: absolute error function and signed error function __where to refer to?__

### Definition of the absolute srror function: 

$\alpha_i = \left| y_i - \hat{y}_i \right| $

Second function is used to form the borders for conformal regressor.

In [5]:
def abserror(prediction_set, y):
    return np.abs(prediction_set - y)


def abs_err_inv(cal_score, significance,method):
    cal_scores_sorted = np.sort(cal_score)[::-1]
    border = (1-significance)*((len(cal_scores_sorted)+1)/len(cal_scores_sorted))
    #border = 1 - significance
    quantile_significance = np.quantile(cal_scores_sorted,border, method=method)
    cal_scores_sorted_bool = cal_scores_sorted >= quantile_significance
    for i in range(len(cal_scores_sorted)):
        if cal_scores_sorted_bool[i]:
            number = i
    return cal_scores_sorted[number]


### Definition of the signed error function: 

$\alpha_i = y_i - \hat{y}_i$

Second function is used to form the borders for conformal regressor.

In [6]:
def sign_error(prediction_set, y):
    return prediction_set - y

def sign_error_inv(cal_score,significance,method):
    cal_scores_sorted = np.sort(cal_score)[::-1]
    quantile_significance_up = np.quantile(cal_scores_sorted, significance / 2, method=method)
    quantile_significance_low = np.quantile(cal_scores_sorted, 1 - significance / 2, method=method)
    up_bool = np.array(cal_scores_sorted <= quantile_significance_up)
    low_bool = np.array(cal_scores_sorted <= quantile_significance_low)
    return -cal_scores_sorted[np.where(up_bool == True)[0][0]], cal_scores_sorted[np.where(low_bool == True)[0][0]]

### Applying error functions on calibration data.

In [7]:
cal_scores_abs = abserror(predictions_cal, data_target[idx_cal])
cal_scores_sign = sign_error(predictions_cal, data_target[idx_cal])

### Generating border values
Border values helps us define upper and lower limits. So future predictions with some significance level must be in a such prediction region. This computation is based on an algorithm for finding the quantile, or a position in a sorted calibration dataset and then comparison test dataset predictions with calibration quantile.  


In [9]:
significance = 0.05

In [15]:
intervals_abs = np.zeros((idx_test.size, 2))
intervals_sign = np.zeros((idx_test.size, 2))

borders_abs = abs_err_inv(cal_scores_abs, significance=significance, method='weibull')
intervals_abs[:, 0] = predictions_test - borders_abs
intervals_abs[:, 1] = predictions_test + borders_abs

borders_sign = sign_error_inv(cal_scores_sign, significance=significance, method='weibull')
intervals_sign[:, 0] = predictions_test - borders_sign[0]
intervals_sign[:, 1] = predictions_test + borders_sign[1]


# Nonconformist library
Now we will use [nonconformist library](https://github.com/donlnz/nonconformist) to show the same result.

In [22]:
nc_sign = NcFactory.create_nc(model, err_func=SignErrorErrFunc())	# Create a default nonconformity function
icp_sign = IcpRegressor(nc_sign)			# Create an inductive conformal regressor

# To avoid overfitting the model - we should comment this line 
#icp_sign.fit(data_data[idx_train, :], data_target[idx_train])

# Calibrate the ICP using the calibration set
icp_sign.calibrate(data_data[idx_cal, :], data_target[idx_cal])

# Produce predictions for the test set, with confidence 95%
prediction_sign = icp_sign.predict(data_data[idx_test, :],significance = significance)

# Print first 5 predictions
prediction_sign[:5]

array([[ 3.1559041,  5.4234862],
       [-0.4115197,  1.8560624],
       [ 1.2780506,  3.5456327],
       [ 0.7563003,  3.0238824],
       [-0.5486697,  1.7189124]])

In [23]:
nc_abs = NcFactory.create_nc(model, err_func=AbsErrorErrFunc())	# Create a default nonconformity function
icp_abs = IcpRegressor(nc_abs)			# Create an inductive conformal regressor

#icp_sign.fit(data_data[idx_train, :], data_target[idx_train])

# Calibrate the ICP using the calibration set
icp_abs.calibrate(data_data[idx_cal, :], data_target[idx_cal])

# Produce predictions for the test set, with confidence 95%
prediction_abs = icp_abs.predict(data_data[idx_test, :], significance = significance)

# Print first 5 predictions
prediction_abs[:5]

array([[ 3.2748138,  5.5725738],
       [-0.29261  ,  2.00515  ],
       [ 1.3969603,  3.6947203],
       [ 0.87521  ,  3.17297  ],
       [-0.42976  ,  1.868    ]])

In [24]:
sign_sum = np.sum(prediction_sign != intervals_sign)
abs_sum = np.sum(prediction_abs != intervals_abs)

print("Number of different predictions between noncomformist and our region\n\
predictor based on nonconformity values.\n\
Absolute error function:{}\n\
Signed error function:{}".format(abs_sum,sign_sum))

Number of different predictions between noncomformist and our region
predictor based on nonconformity values.
Absolute error function:0
Signed error function:0


## Important features of error functions
There are some features to be aware of, because they can make significant impact on the output of conformal regressor. 


**First:** the size of the calibration dataset. The number of points, or the size, must be a multiple of 100, because when we're taking the quantile - different number can be choosen by ***numpy.quantile()*** function.As the function for choosing quantile position in the calibration dataset and function in **nonconformist library** is different we can get various values, but they are not likely to differ very much in numerical value(in most of the cases).
**And the method parameter** of taking the quantile in ***numpy.quantile()*** function can make adjustments [when the desired quantile lies between two data points](https://numpy.org/doc/stable/reference/generated/numpy.quantile.html).
Below there's an example of the synthetic array, but it will be still clear how the **first feature** can affect the output of the ***numpy.quantile()*** function, comparison will be computed with the function from the **nonconformist library.**

In [75]:
arr_mult = np.arange(0,200,1)
arr_nonmult = np.arange(1,192,1)
print("Size of first array:{},\n Size of second array:{}".format(len(arr_mult),len(arr_nonmult)))
arr_sign = np.arange(0.01, 0.99, 0.01) #example for all significance levels (from 1% to 99%)
false_vals = []
false_vals2 = []
for i in arr_sign:
    quantile_arr = np.array(np.quantile(arr_mult, (1-i * ((len(arr_mult) + 1)/len(arr_mult))), method = 'inverted_cdf' ))
    quantile_arr2 = int(np.floor((1-i) * (len(arr_mult) + 1)))-1
    false_vals.append(quantile_arr == quantile_arr2)
print("Number of False values for first array:{}".format(np.sum(np.array(false_vals) == False)))

for i in arr_sign:
    quantile_arr1 = np.quantile(arr_nonmult, 1-i * ((len(arr_nonmult) + 1)/len(arr_nonmult)), method = 'inverted_cdf' )
    quantile_arr21 = int(np.floor((1-i) * (len(arr_nonmult) + 1)))-1
    false_vals2.append(quantile_arr1 == quantile_arr21)
print("Number of False values for second array:{}".format(np.sum(np.array(false_vals2) == False)))

Size of first array:200,
 Size of second array:191
Number of False values for first array:0
Number of False values for second array:96


**As we can see - size of the calibration set must be a multiple of 100. Otherwise - we will recieve wrong values for almost all cases.**

**Second:** correction coefficient. 
Computing not just (1- *significance*) but a significance multiplied by correction coefficient, which helps us adapt to the different sizes of the dataset.

$(1-significance)$

$(1-significance)*(len+1)/len$ , where len is length of the calibration set

However here should be noted, that *method* parameter in ***numpy.quantile()*** function can help us get right values. Below will be example of how the combination of method parameter and significance level estimation.

In [27]:
def abs_err_inv_example(cal_score, significance,method, borders):
    cal_scores_sorted = np.sort(cal_score)[::-1]
    if borders==1:
        border = 1 - significance 
    else:
        border = (1-significance)*((len(cal_scores_sorted)+1)/len(cal_scores_sorted))
    quantile_significance = np.quantile(cal_scores_sorted,border, method=method)
    cal_scores_sorted_bool = cal_scores_sorted >= quantile_significance
    for i in range(len(cal_scores_sorted)):
        if cal_scores_sorted_bool[i]:
            number = i
    return cal_scores_sorted[number]

In [77]:
method_q = ['inverted_cdf','averaged_inverted_cdf','closest_observation','interpolated_inverted_cdf',
              'hazen','weibull','median_unbiased','normal_unbiased']
intervals_abs1 = np.zeros((idx_test.size, 2))
intervals_abs2 = np.zeros((idx_test.size, 2))
a = []
b = []
print("\nSignificance estimation + method and number of different predictions\n")
for i in method_q:
    borders_abs = abs_err_inv_example(cal_scores_abs, significance=significance, method=i, borders = 1)
    intervals_abs1[:, 0] = predictions_test - borders_abs
    intervals_abs1[:, 1] = predictions_test + borders_abs
    abs_sum1 = np.sum(prediction_abs != intervals_abs1)
    #print("\nNormal significance level estimation + {} \n\
    #Number of different predictions:{}".format(i,abs_sum1))
    print(i,abs_sum1)
    a.append(abs_sum1)
    borders_abs2 = abs_err_inv_example(cal_scores_abs, significance=significance, method=i, borders = 2)
    intervals_abs2[:, 0] = predictions_test - borders_abs2
    intervals_abs2[:, 1] = predictions_test + borders_abs2
    abs_sum2 = np.sum(prediction_abs != intervals_abs2)
    #print("\nSignificance level estimation multiplied by correction coefficient + {} \n\
    #Number of different predictions:{}".format(i,abs_sum2))
    print(i,abs_sum2)
    b.append(abs_sum2)

print("\n Average number of non-equal predictions for all methods \n\
and for normal significance level:{}\n\
 Average number of non-equal predictions for all methods \n\
and for significance level multiplied by correction coefficient:{}".format(np.sum(a)/len(method_q),np.sum(b)/len(method_q)))


Significance estimation + method and number of different predictions

inverted_cdf 4128
inverted_cdf 0
averaged_inverted_cdf 0
averaged_inverted_cdf 0
closest_observation 4128
closest_observation 0
interpolated_inverted_cdf 4128
interpolated_inverted_cdf 0
hazen 0
hazen 4128
weibull 0
weibull 4128
median_unbiased 0
median_unbiased 4128
normal_unbiased 0
normal_unbiased 4128

 Average number of non-equal predictions for all methods 
and for normal significance level:1548.0
 Average number of non-equal predictions for all methods 
and for significance level multiplied by correction coefficient:2064.0
